In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d harshghadiya/kidneystone
!unzip \*.zip && rm *.zip


  0% 0.00/1.89k [00:00<?, ?B/s]
100% 1.89k/1.89k [00:00<00:00, 4.90MB/s]
Archive:  kidneystone.zip
  inflating: kidney-stone-dataset.csv  


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Step 1: Load Data
data = pd.read_csv('/content/kidney-stone-dataset.csv')  # Replace '/path/to/your/dataset.csv' with the actual path

# Step 2: Prepare Data
X = data[['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc']]
y = data['target']

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Find mutual k-nearest neighbors
k = 5  # Number of neighbors
nn_model = NearestNeighbors(n_neighbors=k)
nn_model.fit(X_train_scaled)
distances, indices = nn_model.kneighbors(X_train_scaled)

# Step 6: Create a mutual k-nearest neighbor graph
mutual_indices = []
for i, neighbors in enumerate(indices):
    mutual_neighbors = np.intersect1d(neighbors, np.where(indices[neighbors] == i))
    mutual_indices.append(mutual_neighbors)

# Step 7: Train the MKNN model
# Here you can use any classification algorithm with mutual_indices as the neighbors
# For demonstration, let's use KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
mknn_model = KNeighborsClassifier(n_neighbors=k)
mknn_model.fit(X_train_scaled, y_train)

# Step 8: Predict using the trained MKNN model
y_pred = mknn_model.predict(X_test_scaled)

# Step 9: Evaluate the MKNN model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Display evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Display classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.6111111111111112
Precision: 0.6277056277056277
Recall: 0.6111111111111112
F1 Score: 0.6176142697881828

Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.67      0.70        12
           1       0.43      0.50      0.46         6

    accuracy                           0.61        18
   macro avg       0.58      0.58      0.58        18
weighted avg       0.63      0.61      0.62        18

